# Creation of the Vector Database to store MtG Card Images

We will be using Pinecone to create a vector database that stores the MtG card image embeddings.

In [10]:
import os
import numpy as np
import pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import cv2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input

# Load the EfficientNetB0 model pre-trained on ImageNet, excluding the top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

# Pinecone Initialization (new method)
api_key = os.getenv("PINECONE_API_KEY")  # Get your API key from environment variables
pinecone_client = pinecone.Pinecone(api_key=api_key)

# Check existing indexes
indexes = pinecone_client.list_indexes().names()  # Returns a list of existing index names
print(f"Existing indexes: {indexes}")

# Connect to a Pinecone index
index_name = 'mtg-cards-index-efficientnet'
if index_name not in indexes:
    print(f"Creating index: {index_name}")
    pinecone_client.create_index(
        name=index_name, 
        dimension=1280,  # Dimension matches the EfficientNetB0 output
        metric='cosine', 
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1',
        )
    )
else:
    print(f"Index {index_name} already exists. Skipping creation.")
    
index = pinecone_client.Index(index_name)

def resize_with_padding(img, target_size):
    h, w, _ = img.shape
    scale = min(target_size / h, target_size / w)
    new_w = int(w * scale)
    new_h = int(h * scale)
    resized_img = cv2.resize(img, (new_w, new_h))
    padded_img = np.full((target_size, target_size, 3), 255, dtype=np.uint8)
    pad_w = (target_size - new_w) // 2
    pad_h = (target_size - new_h) // 2
    padded_img[pad_h:pad_h + new_h, pad_w:pad_w + new_w, :] = resized_img
    return padded_img

def preprocess_image_with_padding(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    padded_img = resize_with_padding(img, target_size[0])
    padded_img = image.img_to_array(padded_img)
    padded_img = np.expand_dims(padded_img, axis=0)
    padded_img = preprocess_input(padded_img)
    return padded_img

def embed_image(img_path):
    img_array = preprocess_image_with_padding(img_path)
    embedding = base_model.predict(img_array)
    return embedding.flatten()

def batch_upsert_embeddings(image_dir, batch_size=100, namespace="cards"):
    batch = []
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        try:
            # Generate the embedding
            embedding = embed_image(img_path)
            # Add the vector to the batch with the image name as the ID
            batch.append({"id": img_name, "values": embedding.tolist()})
            # Upsert the batch when it reaches the specified size
            if len(batch) >= batch_size:
                index.upsert(vectors=batch, namespace=namespace)
                print(f"Upserted batch of {batch_size} vectors.")
                batch.clear()  # Clear the batch
        except Exception as e:
            print(f"Error processing image {img_name}: {e}")
    
    # Upsert any remaining vectors in the last batch
    if batch:
        index.upsert(vectors=batch, namespace=namespace)
        print(f"Upserted final batch of {len(batch)} vectors.")



Existing indexes: ['mtg-cards-index-efficientnet']
Index mtg-cards-index-efficientnet already exists. Skipping creation.


In [13]:
# Directory where images are saved
image_dir = "Datasets/mtg_images"

# Run the batch upsert
batch_upsert_embeddings(image_dir, batch_size=100, namespace="mtg_cards")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 

In [41]:
stats = index.describe_index_stats()
print(stats)

{'dimension': 1280,
 'index_fullness': 0.0,
 'namespaces': {'mtg_cards': {'vector_count': 95486}},
 'total_vector_count': 95486}


## Testing a picture from my phone and IT WORKS

The top 2 results are the card I uploaded

In [42]:
test = embed_image('Datasets/mtg_test_images/shuko.jpg')

# Convert the NumPy array to a list
test_list = test.tolist()

query_results = index.query(
    namespace="mtg_cards",
    vector=test_list,  # Pass the list instead of the NumPy array
    top_k=3
)

print(query_results)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
{'matches': [{'id': 'a47456b8-cef8-4085-90b1-92788e16fd27.jpg',
              'score': 0.930829465,
              'values': []},
             {'id': 'a88ed77e-5eae-4282-8f19-75dc8437f83d.jpg',
              'score': 0.919052958,
              'values': []},
             {'id': '011bc5b7-c4d5-4c4c-af0d-aa0853d63f3a.jpg',
              'score': 0.815545797,
              'values': []}],
 'namespace': 'mtg_cards',
 'usage': {'read_units': 6}}


# Now get result, find the price, and add to a CSV

## Scryfall API Call

In [50]:
# Now query the Scryfall API to get the card details
import requests

# Remove the file extension from the top result ID so we can get the id of the card

def get_card_id(query_results):
    return query_results.matches[0].id.split('.')[0]

def get_card_details(card_id):
    scryfall_api_url = f"https://api.scryfall.com/cards/{card_id}"
    response = requests.get(scryfall_api_url)
    card_data = response.json()
    return card_data

print(get_card_id(query_results))


a47456b8-cef8-4085-90b1-92788e16fd27


In [51]:
card_data = get_card_details(top_result_id)

print(card_data)
print(card_data['name'])
print(card_data['prices']['usd'])


{'object': 'card', 'id': 'a47456b8-cef8-4085-90b1-92788e16fd27', 'oracle_id': '8abe0577-8fdb-4e4a-a871-01b21732c961', 'multiverse_ids': [74415], 'mtgo_id': 21691, 'mtgo_foil_id': 21692, 'tcgplayer_id': 12351, 'cardmarket_id': 12909, 'name': 'Shuko', 'lang': 'en', 'released_at': '2005-02-04', 'uri': 'https://api.scryfall.com/cards/a47456b8-cef8-4085-90b1-92788e16fd27', 'scryfall_uri': 'https://scryfall.com/card/bok/159/shuko?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_status': 'highres_scan', 'image_uris': {'small': 'https://cards.scryfall.io/small/front/a/4/a47456b8-cef8-4085-90b1-92788e16fd27.jpg?1562878892', 'normal': 'https://cards.scryfall.io/normal/front/a/4/a47456b8-cef8-4085-90b1-92788e16fd27.jpg?1562878892', 'large': 'https://cards.scryfall.io/large/front/a/4/a47456b8-cef8-4085-90b1-92788e16fd27.jpg?1562878892', 'png': 'https://cards.scryfall.io/png/front/a/4/a47456b8-cef8-4085-90b1-92788e16fd27.png?1562878892', 'art_crop': 'https://cards.scryfall.io/art_

In [56]:
import csv
import os

def add_card_data_to_csv(file_path, top_result_id):
    # Read the existing CSV file into a list of dictionaries
    if os.path.isfile(file_path):
        with open(file_path, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
    else:
        rows = []

    card_data = get_card_details(top_result_id)

    # Extract the relevant data from the card_data dictionary
    id = card_data['id']
    name = card_data['name']
    mana_cost = card_data.get('mana_cost', '')
    cmc = card_data.get('cmc', 0)
    type_line = card_data.get('type_line', '').replace('—', '-')
    colors = card_data.get('colors', [])
    color_identity = card_data.get('color_identity', [])
    set_name = card_data.get('set_name', '')
    rarity = card_data.get('rarity', '')
    full_art = card_data.get('full_art', False)
    price = card_data.get('prices', {}).get('usd', '')
    number_owned = 0
    total_value = 0.0

    # Check if a row with the same id already exists
    row_exists = False
    for row in rows:
        if row['id'] == id:
            # Update the number_owned and total_value
            row['number_owned'] = int(row['number_owned']) + 1
            row['total_value'] = row['number_owned'] * float(price)
            row_exists = True
            break

    if not row_exists:
        # Add a new row
        new_row = {
            "id": id,
            "name": name,
            "mana_cost": mana_cost,
            "cmc": cmc,
            "type_line": type_line,
            "colors": colors,
            "color_identity": color_identity,
            "set_name": set_name,
            "rarity": rarity,
            "full_art": full_art,
            "price": price,
            "number_owned": 1,
            "total_value": float(price)
        }
        rows.append(new_row)

    # Write the updated data back to the CSV file
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ["id", "name", "mana_cost", "cmc", "type_line", "colors", "color_identity", "set_name", "rarity", "full_art", "price", "number_owned", "total_value"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerows(rows)

file_path = 'Output/mtg_catalog.csv'

add_card_data_to_csv(file_path, top_result_id)
print(f"Rows added to {file_path}.")

Rows added to Output/mtg_catalog.csv.
